In [84]:
import numpy as np
import random
import time
from functools import wraps
from typing import Callable
from warnings import warn

Wskaźnik uwarunkowania macierzy

In [12]:
def norm(A : np.ndarray) -> float:
    n = len(A)
    return max(sum(A[i][j] for j in range(n)) for i in range(n))

In [13]:
def conditioning_factor(A : np.ndarray) -> np.array:
    A_inv = np.linalg.inv(A)
    return norm(A_inv) * norm(A)

Time calculation

In [107]:
def measure_time(func : Callable) -> Callable:
    
    @wraps(func)
    def wrapper(*args) -> tuple[np.ndarray, float]:
        
        start = time.perf_counter()
        
        X_approx = func(*args)
        
        end = time.perf_counter()

        return X_approx, end - start
    return wrapper

Gaussian Elimination

![title](img_vsc/img01.png)

![title](img_vsc/img02.png)

In [108]:
@measure_time
def gaussian_elimination(A : np.ndarray, B : np.ndarray) -> np.ndarray:
    
    n = A.shape[0]
    
    C = np.hstack((A, B.reshape(n, 1)))


    for i in range(n):
        for j in range(i + 1, n):
            ratio = C[j][i] / C[i][i]
            C[j] = C[j] - ratio * C[i]

    X = np.zeros(n, dtype=A.dtype)

    X[n - 1] = C[n - 1][n] / C[n - 1][n - 1] # We now have n + 1 columns

    for i  in range(n - 2 , -1, -1):
        X[i] = (C[i][n] - sum(X[i+1:n]*C[i][i+1:n])) / C[i][i]
        
    return X

Algorytm Thomasa dla macierzy trójdiagonalnej

In [169]:
@measure_time
def thomas_algorithm(A : np.ndarray, B : np.ndarray, C : np.ndarray, D : np.ndarray) -> np.ndarray:
    
    N = len(B)
    
    C[0] /= B[0]
    D[0] /= B[0]
    
    for i in range(1, N - 1):
        
        common_coefficient = B[i] - A[i - 1] * C[i - 1]
        
        C[i] /= common_coefficient
        D[i] = (D[i] - A[i - 1] * D[i - 1])  / common_coefficient
        
    # Since loop goes only to n - 1, we need to calculate the last element of D
    D[N - 1] = (D[N - 1] - A[N - 2] * D[N - 2]) / (B[N - 1] - A[N - 2] * C[N - 2])
    
    # Backward substitution
    X = [0] * N
    X[-1] = D[-1]
    
    for i in range(N - 2, -1, -1):
        X[i] = D[i] - C[i] * X[i + 1]
        
    return X    

In [164]:
class GaussTest():
    
    def __init__(self, get_matrix_function : Callable):

        self.get_matrix = get_matrix_function


    def random_vector(self, n : int) -> list[float]:
        
        return [-1 if random.random() < 0.5 else 1 for _ in range(n)]
        
    def get_system(self, n : int, data_type : np.float_) -> list[np.ndarray, np.ndarray, np.ndarray]:
        
        A = np.array(self.get_matrix(n), dtype=data_type)
        X = np.array(self.random_vector(n), dtype=data_type)
        
        return A, A @ X, X
        
    def solve_gaussian(self, A : np.ndarray, B : np.ndarray) -> tuple[np.ndarray, np.ndarray, float, float]:

        X_approx, duration = gaussian_elimination(A, B)
        
        cond = None
        
        try:
            cond = conditioning_factor(A)
            
        except:
            warn(Warning("Conditioning factor is not supported for given float precision"))
        
        return X_approx, duration, cond
    
    
    def solve_n(self, n_min : int, n_max : int, data_types : list[np.float_]) -> list[list[tuple[int, float, float, float]]]:
        
        results = [[] for _ in range(len(data_types))]

        for n in range(n_min, n_max + 1):

            A = self.get_matrix(n)   
            X = self.random_vector(n)

            for i, data_type in enumerate(data_types):

                A = np.array(A, dtype=data_type).reshape(n, n)
                X = np.array(X, dtype=data_type).reshape(n)

                B = A @ X

                X_approx, duration, cond = self.solve_gaussian(A, B)
                    
                results[i].append([n, duration, cond, np.linalg.norm(X - X_approx)])
            
        return results
    
    @staticmethod
    def thomasify_matrix(A : np.ndarray, data_type : np.float_) -> tuple[np.ndarray, np.ndarray, np.ndarray]:

        a = []
        b = []
        c = []

        for i in range(len(A)):
            if i + 1 < len(A):
                a.append(A[i + 1][i])
                c.append(A[i][i + 1])
            b.append(A[i][i])

        a = np.array(a, dtype=data_type).reshape(len(a))
        b = np.array(b, dtype=data_type).reshape(len(b))
        c = np.array(c, dtype=data_type).reshape(len(c))

        return a, b, c


    def solve_n_gauss_vs_thomas(self, n_min : int, n_max : int, data_types : list[np.float_]) -> list[list[tuple[int, float, float, float]]]:

        results = [ [] for i in range(len(data_types))]

        for n in range(n_min, n_max + 1):

            if n % 50 == 0:
                print(n)

            A = self.get_matrix(n)
            X = self.random_vector(n)

            for i, data_type in enumerate(data_types):

                # Gaussian section

                A = np.array(A, dtype=data_type).reshape(n, n)
                X = np.array(X, dtype=data_type).reshape(n)

                B = A @ X
 
                gauss_X_approx, gauss_duration, cond = self.solve_gaussian(A, B)

                # Thomas section
                a, b, c = GaussTest.thomasify_matrix(A, data_type)

                thomas_X_approx, thomas_duration = thomas_algorithm(a, b, c, B)
                
                results[i].append([n,gauss_duration, thomas_duration, cond, np.linalg.norm(X - gauss_X_approx), np.linalg.norm(X - thomas_X_approx)])

        return results

In [97]:
def zad1_matrix(n : int) -> list[list]:
    
    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]
        
    for i in range(1, n + 1):
        for j in range(1, n + 1):
                
            if i == 1:
                A[i - 1][j - 1] = 1
            else:
                A[i - 1][j - 1] =  1 / (i + j - 1)
                    
    return A

Examples

In [114]:
zad1 = GaussTest(zad1_matrix)

A, B, X = zad1.get_system(5, np.float32)

X_approx, duration, cond = zad1.solve_gaussian(A, B)
print(X_approx, duration, cond)

[ 0.99988645  1.0010474   0.9970335  -0.99668515  0.99871784] 0.00011499999891384505 28023.636474609375


In [115]:
zad1 = GaussTest(zad1_matrix)

A, B, X = zad1.get_system(5, np.float64)

X_approx, duration, cond = zad1.solve_gaussian(A, B)
print(X_approx, duration, cond)

[-1. -1. -1. -1.  1.] 0.0003003000019816682 28000.000000047658


Tests

In [125]:
zad1 = GaussTest(zad1_matrix)
zad1_results = zad1.solve_n(1, 30, [np.float32, np.float64])

In [126]:
zad1_results_float32 = zad1_results[0]

for n, duration, conditioning, error in zad1_results_float32:
    print(n, duration, conditioning, error)

1 3.140000262646936e-05 1.0 0.0
2 3.610000203480013e-05 8.000000476837158 0.0
3 6.479999865405262e-05 216.00041484832764 0.0
4 7.879999975557439e-05 2880.032928466797 0.00032981503
5 7.959999857121147e-05 28023.636474609375 0.0058726547
6 0.000185500000952743 232606.4384765625 0.08523364
7 0.0003079999987676274 2898755.69921875 0.45904684
8 0.00014240000018617138 1316895.181640625 3.4430351
9 0.0003176000027451664 14008956.3984375 25.438406
10 0.0002976000032504089 4207927.08984375 7.035705
11 0.0003880000003846362 6356634.32421875 22.072447
12 0.00024000000121304765 2712885.1083984375 56.687496
13 0.0002994999995280523 2723394.9462890625 1984.4052
14 0.0005023000012442935 3962083.2719726562 25.03631
15 0.0003953000014007557 20024231.909179688 35.653778
16 0.001092000002245186 19575512.140625 15.985135
17 0.0004977999997208826 25471486.899414062 19.029455
18 0.0005110999991302378 16330873.974609375 25.31294
19 0.0006063000000722241 7791152.4873046875 20.441757
20 0.0008358000013686251 

In [127]:
zad1_results_float64 = zad1_results[1]

for n, exec_time, conditioning, error in zad1_results_float64:
    print(n, exec_time, conditioning, error)

1 1.4599998394260183e-05 1.0 0.0
2 3.1600000511389226e-05 8.00000035762789 0.0
3 5.859999873791821e-05 216.0003991134734 0.0
4 5.880000026081689e-05 2880.032476674358 0.0
5 0.00011010000162059441 28023.64869388366 2.733112487701844e-13
6 0.00018849999833037145 232606.2172930967 2.195168041710574e-12
7 0.0002094999981636647 2898707.638811715 3.2366673500155362e-09
8 0.00012210000204504468 1316884.9719799757 5.646390511983806e-09
9 0.00015109999731066637 14009484.881372094 3.293999039764078e-08
10 0.00015959999655024149 4208044.049106911 2.291844134731236e-09
11 0.00030360000164364465 6356489.009672537 1.2514402777711886e-08
12 0.00021949999791104347 2712841.002320409 5.825025152003579e-09
13 0.00039060000199242495 2723570.7333032563 1.4839322905910675e-08
14 0.00029100000028847717 3962179.0572766066 8.039183682699212e-09
15 0.0005387999990489334 20024346.985491514 7.004506024837158e-09
16 0.0010588000004645437 19576032.14162445 4.2999950658050035e-08
17 0.0004259000015736092 25471425.69

In [129]:
def zad2_matrix(n : int) -> np.ndarray:
    
    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]
    
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            
            if j >= i:
                A[i - 1][j - 1] = 2 * i / j
            else:
                A[i - 1][j - 1] = A[j - 1][i - 1]
                
    return A

Examples

In [132]:
zad2 = GaussTest(zad2_matrix)

A, B, X = zad1.get_system(5, np.float64)

X_approx, exec_time, cf = zad2.solve_gaussian(A, B)
print(X, X_approx, exec_time, cf)

[ 1. -1.  1.  1.  1.] [ 1. -1.  1.  1.  1.] 0.00017869999282993376 28000.000000047658


In [134]:
zad2 = GaussTest(zad2_matrix)

A, B, X = zad1.get_system(5, np.float64)

X_approx, exec_time, cf = zad2.solve_gaussian(A, B)
print(X, X_approx, cf)

[-1.  1.  1.  1. -1.] [-1.  1.  1.  1. -1.] 28000.000000047658


Tests

In [144]:
zad2 = GaussTest(zad2_matrix)
zad2_results = zad2.solve_n(1, 500, [np.float32, np.float64])

In [145]:
zad2_results_float32 = zad2_results[0]
for n, exec_time, conditioning, error in zad2_results_float32:
    print(n, exec_time, conditioning, error)

1 4.7799992898944765e-05 1.0 0.0
2 6.800000119255856e-05 1.0000000298023224 0.0
3 4.669999907491729e-05 1.4444445007377202 2.9200194e-07
4 0.00011670000094454736 1.8333334078391397 1.6858739e-07
5 7.869999535614625e-05 2.233333435654641 1.3328004e-07
6 0.0001649999976507388 2.6444445444477935 1.0237434e-06
7 0.00022900000476511195 3.031746150009214 2.3398145e-06
8 0.0002615999983390793 3.4484128290935177 5.177033e-06
9 0.000342100000125356 3.849206492777855 2.3669875e-06
10 0.00025129999994533136 4.2492065205933525 4.64381e-06
11 0.00020840000070165843 4.6594277916181355 4.5794836e-06
12 0.00024370000028284267 5.055219045933651 3.0584793e-06
13 0.0002958999975817278 5.465475483699897 5.6702784e-06
14 0.0006152999994810671 5.868897985485128 9.920727e-06
15 0.0007832999981474131 6.268898013300629 7.3835217e-06
16 0.0008375999968848191 6.678405180923292 1.2429534e-05
17 0.0004537999993772246 7.077618273425637 1.7090222e-05
18 0.0005100999987917021 7.485025688559575 1.8817613e-05
19 0.0005

In [146]:
zad2_results_float64 = zad2_results[1]
for n, exec_time, conditioning, error in zad2_results_float64:
    print(n, exec_time, conditioning, error)

1 2.130000211764127e-05 1.0 0.0
2 3.60000049113296e-05 1.0 0.0
3 4.25000034738332e-05 1.444444457689921 0.0
4 0.00011790000280598179 1.8333333532015479 0.0
5 5.5500000598840415e-05 2.233333369096121 2.482534153247273e-16
6 0.00016090000281110406 2.644444465637208 6.377745716588144e-16
7 9.219999628840014e-05 3.0317460596561405 8.08254562088053e-16
8 0.0002472000051056966 3.44841272632281 2.1784148785229476e-15
9 0.00025179999647662044 3.849206378062566 1.5100665727558131e-15
10 0.0001616000008652918 4.2492063939571345 2.169910949333616e-15
11 0.0003853999951388687 4.659427652756371 2.9914279021231404e-15
12 0.00023890000011306256 5.055218895276388 6.237042840057389e-15
13 0.0006986000007600524 5.46547532081604 6.561596535353689e-15
14 0.0006317000006674789 5.868897810578344 1.3496403720850575e-14
15 0.00036830000317422673 6.268897826472916 1.6250267321577627e-14
16 0.0005218999940552749 6.678404981891315 1.552209318316311e-14
17 0.0004134999981033616 7.077618062496182 1.617766767054864

In [148]:
def zad3a_matrix(n : int) -> np.ndarray:

    k = 6
    m = 3

    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]

    for i in range(1, n + 1):
        for j in range(1, n + 1):

            if j == i:
                A[i - 1][j - 1] = k
            elif j == i + 1:
                A[i - 1][j - 1] = 1 / (i + m)
            elif i > 1 and j == i - 1:
                A[i - 1][j - 1] = k / (i + m + 1)

    return A

Examples

In [149]:
zad3a = GaussTest(zad3a_matrix)

A, B, X = zad1.get_system(5, np.float32)

X_approx, exec_time, cf = zad3a.solve_gaussian(A, B)
print(X, X_approx, exec_time, cf)

[-1.  1. -1.  1. -1.] [-0.99999994  0.9999696  -0.9998529   0.9997846  -0.99990135] 0.00017179999849759042 28023.636474609375


In [150]:
zad3a = GaussTest(zad3a_matrix)

A, B, X = zad1.get_system(5, np.float64)

X_approx, exec_time, cf = zad3a.solve_gaussian(A, B)
print(X, X_approx, exec_time, cf)

[ 1. -1. -1. -1. -1.] [ 1. -1. -1. -1. -1.] 9.700000373413786e-05 28000.000000047658


In [151]:
def zad3b_matrix(n : int) -> np.ndarray:

    k = 6
    m = 3

    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]

    for i in range(1, n + 1):
        for j in range(1, n + 1):
            
            if j == i:
                A[i - 1][j - 1] = -m * i - k
            elif j == i + 1:
                A[i - 1][j - 1] = i
            elif i > 1 and j == i - 1:
                A[i - 1][j - 1] = m / i

    return A

Examples

In [152]:
zad3b = GaussTest(zad3b_matrix)

A, B, X = zad1.get_system(5, np.float32)

X_approx, exec_time, cf = zad3b.solve_gaussian(A, B)
print(X, X_approx, exec_time, cf)

[ 1. -1. -1.  1. -1.] [ 1.000009   -1.0000687  -0.9998261   0.99982005 -0.99993426] 0.00012820000119972974 28023.636474609375


In [153]:
zad3b = GaussTest(zad3b_matrix)

A, B, X = zad1.get_system(5, np.float64)

X_approx, exec_time, cf = zad3b.solve_gaussian(A, B)
print(X, X_approx, exec_time, cf)

[ 1. -1.  1.  1. -1.] [ 1. -1.  1.  1. -1.] 9.539999882690609e-05 28000.000000047658


Thomasify example

In [155]:
A = np.array([[10,2,0,0],
              [3,10,4,0],
              [0,1,7,5],
              [0,0,3,4]],dtype=float)   

a, b, c = GaussTest.thomasify_matrix(A, np.float64)
print(a, b, c)

[3. 1. 3.] [10. 10.  7.  4.] [2. 4. 5.]


In [170]:
zad3a = GaussTest(zad3a_matrix)

zad3a_results = zad3a.solve_n_gauss_vs_thomas(3, 500, [np.float32, np.float64])

50


C:\Users\48667\AppData\Local\Temp\ipykernel_8344\291966129.py:29: Warning: Conditioning factor is not supported for given float precision
  warn(Warning("Conditioning factor is not supported for given float precision"))


100
150
200
250
300
350
400
450
500


In [171]:
zad3_results_float32 = zad3a_results[0]

for n, gauss_exec_time, thomas_exec_time, conditioning, gauss_error, thomas_error in zad3_results_float32:
    print(n, gauss_exec_time, thomas_exec_time, conditioning, gauss_error, thomas_error)

3 0.00012910000077681616 1.3499993656296283e-05 1.159526202620794 1.3328004e-07 1.3328004e-07
4 0.00010750000365078449 1.0399999155197293e-05 1.1594843539359265 0.0 0.0
5 9.29000016185455e-05 9.79999458650127e-06 1.159485363403662 1.3328004e-07 1.3328004e-07
6 0.00016999999934341758 2.149999636458233e-05 1.1594853421668128 1.3328004e-07 1.3328004e-07
7 0.00030989999504527077 3.369999467395246e-05 1.159485342565811 1.4600097e-07 1.5769906e-07
8 0.0002708999963942915 2.1699997887481004e-05 1.159485342559107 1.3328004e-07 1.3328004e-07
9 0.00016830000095069408 1.4700002793688327e-05 1.1594853425592093 2.4575624e-07 2.308478e-07
10 0.00017470000602770597 1.979999797185883e-05 1.1594853425592078 1.1920929e-07 1.4600097e-07
11 0.0002086000022245571 3.189999551977962e-05 1.1594853425592078 1.6858739e-07 2.0647654e-07
12 0.0003001000004587695 4.099999932805076e-05 1.1594853425592078 1.8848644e-07 1.3328004e-07
13 0.000836799998069182 1.9899998733308166e-05 1.1594853425592078 1.4600097e-07 1.33

In [172]:
zad3_results_float64 = zad3a_results[1]

for n, gauss_exec_time, thomas_exec_time, conditioning, gauss_error, thomas_error in zad3_results_float64:
    print(n, gauss_exec_time, thomas_exec_time, conditioning, gauss_error, thomas_error)

3 7.100000220816582e-05 1.3199998647905886e-05 1.15952620005856 0.0 0.0
4 5.070000042906031e-05 8.299997716676444e-06 1.1594843879478562 0.0 2.220446049250313e-16
5 6.320000102277845e-05 1.5900004655122757e-05 1.1594853988848859 0.0 2.220446049250313e-16
6 0.00017410000145900995 1.7300000763498247e-05 1.1594853775722198 0.0 2.482534153247273e-16
7 9.610000415705144e-05 2.390000008745119e-05 1.1594853779708043 3.1401849173675503e-16 1.1102230246251565e-16
8 0.00023009999858913943 1.5600002370774746e-05 1.159485377964106 2.220446049250313e-16 2.220446049250313e-16
9 0.00013390000094659626 1.479999627918005e-05 1.1594853779642083 2.220446049250313e-16 2.482534153247273e-16
10 0.00017579999985173345 1.5099998563528061e-05 1.1594853779642067 2.220446049250313e-16 1.1102230246251565e-16
11 0.00038409999979194254 1.720000000204891e-05 1.1594853779642067 3.1401849173675503e-16 0.0
12 0.0002934000003733672 2.890000178012997e-05 1.1594853779642067 1.1102230246251565e-16 2.482534153247273e-16
13 